# Introduction/Business Problem

**In Toronto, if someone is looking to open a restaurant, where would you recommend that they open it? Similarly, if a contractor is trying to start their own business, where would you recommend that they setup their office?**

# Used Data

**We will use Data from Wikipedia to find the Postal Codes for each Neighborhood and the Geospatial data (https://cocl.us/Geospatial_data) to find the Latitude and Longitude for each Neighbohood.**